In [1]:
import numpy as np
import random 
import math
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

In [11]:
data = pd.read_csv('time_series_covid_19_confirmed.csv')
data = data.drop(['Province/State', 'Lat', 'Long'], axis = 1)
data.head(20) 

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967
1,Albania,0,0,0,0,0,0,0,0,0,...,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402
2,Algeria,0,0,0,0,0,0,0,0,0,...,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273
3,Andorra,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,Angola,0,0,0,0,0,0,0,0,0,...,172,176,183,186,189,197,212,212,259,267
5,Antigua and Barbuda,0,0,0,0,0,0,0,0,0,...,26,26,26,26,26,26,65,65,65,69
6,Argentina,0,0,0,0,0,0,0,0,0,...,39570,41204,42785,44931,47203,49851,52457,55343,57744,59933
7,Armenia,0,0,0,0,0,0,0,0,0,...,19157,19708,20268,20588,21006,21717,22488,23247,23909,24645
8,Australia,0,0,0,0,0,0,0,0,0,...,108,108,108,108,108,108,108,108,108,108
9,Australia,0,0,0,0,3,4,4,4,4,...,3144,3149,3151,3150,3159,3162,3168,3174,3177,3184


In [12]:
data = data.groupby(['Country/Region']).sum()
data.head(10)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,27878,28424,28833,29157,29481,29640,30175,30451,30616,30967
Albania,0,0,0,0,0,0,0,0,0,0,...,1838,1891,1962,1995,2047,2114,2192,2269,2330,2402
Algeria,0,0,0,0,0,0,0,0,0,0,...,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273
Andorra,0,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
Angola,0,0,0,0,0,0,0,0,0,0,...,172,176,183,186,189,197,212,212,259,267
Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,...,26,26,26,26,26,26,65,65,65,69
Argentina,0,0,0,0,0,0,0,0,0,0,...,39570,41204,42785,44931,47203,49851,52457,55343,57744,59933
Armenia,0,0,0,0,0,0,0,0,0,0,...,19157,19708,20268,20588,21006,21717,22488,23247,23909,24645
Australia,0,0,0,0,4,5,5,6,9,9,...,7411,7461,7474,7492,7521,7558,7595,7601,7686,7764
